In [2]:
import pandas as pd
data1 = pd.read_csv("/data/CostarExport_1.csv") # getting the data
data2 = pd.read_csv("/data/LA_multifamily.csv") # getting the data

columnsInD1 = data1.keys()
columnsInD2 = data2.keys()

print(data1.shape)
print(data2.shape)

(472, 246)
(1661, 245)


In [3]:
validCols = ['Property Address', 'Property City', 'Property State',
       'Property County', 'Property Zip Code', 'Property Type', 'Sale Price',
       'Number Of Units', 'Size', 'Property Name',
       'Sale Date', 'Recording Date', 'Hold Period',
       'Actual Cap Rate', 'Pro Forma Cap Rate', 'Asking Price',
       'Down Payment', 'Net Income', 'Price Per Unit',
       'Price Per AC Land', 'Price Per SF', 'Price Per SF (Net)',
       'Price Per SF Land', 'Total Expense Amount', 
       'GIM', 'GRM', 'Amenities', 'Star Rating',
       'Affordable Type', 'Secondary Type', 'Avg Unit SF', 'Market',
       'Submarket Name', 'Location Type', 'Age', 'Building Class',
       'Land Area AC', 'Land Area SF', 'Latitude', 'Longitude',
       'Typical Floor (SF)', 'Number Of Floors', 'Number Of Parking Spaces',
       'Parcel Number 1 (Min)', 'Parcel Number 2 (Max)', 'Tenancy', 'Vacancy',
       'Zoning', 'Studio Mix', 'One Bedroom Mix', 'Two Bedroom Mix',
       'Three Bedroom Mix', 'Other Mix', 'Building SF',
       'Number Of 1 Bedrooms Units', 'Number Of 2 Bedrooms Units', 'Floor Area Ratio', 
       'Assessed Land','Number Of Studios Units', 
       'Number Of 3 Bedrooms Units', 'Year Built']

mixed = pd.merge(data2, data1,
                 on = validCols,
                how = 'outer')

merged_file = r'/data/merged_data.csv'

mixed.to_csv(merged_file, index = False)
pd.DataFrame(mixed)
mixed.head()

,Property Address,Property City,Property State,Property Type,Building SF,Star Rating,Sale Price,Price Per SF,Sale Date,Sale Status_x,...,Sprinklers_y,Sprinklers.1_y,Stamp_y,Submarket Cluster_y,Submarket Code_y,Title Company_y,Transfer Tax_y,Units Per Acre_y,Water_y,Year Renovated_y
0,6319-6323 Primrose Ave,Los Angeles,CA,Multi-Family,"6,156",2 Star,"$1,900,000",308.64,9/15/2023,Sold,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2267-73 1/2 W. Adams Bl.,Los Angeles,CA,Multi-Family,"9,500",2 Star,"$2,400,000",252.63,9/14/2023,Sold,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2677 Waverly Dr,Los Angeles,CA,Multi-Family,"3,890",2 Star,"$2,100,000",539.85,9/13/2023,Sold,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1044 S Orange Grove Ave,Los Angeles,CA,Multi-Family,"5,322",2 Star,"$2,475,000",465.05,9/12/2023,Sold,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2101 Bonsallo Ave,Los Angeles,CA,Multi-Family,"8,718",3 Star,"$1,625,000",186.40,9/12/2023,Sold,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# imports

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

# cleanup
df = pd.read_csv(merged_file) 

columns_to_clean = ["Sale Price", "Net Income", "Building SF", 
                    "Star Rating", "Price Per AC Land",
                    "Price Per Unit", "Down Payment", "Size", "Total Expense Amount",
                    'Parcel Number 1 (Min)', 'Parcel Number 2 (Max)', "Price Per SF Land"]
                    
for col in columns_to_clean:
    df[col] = df[col].str.replace(',', '')
    df[col] = df[col].str.replace('$', '')
    df[col] = df[col].str.replace(' Star', '')
    df[col] = df[col].str.replace('-', '')
    

# Convert the columns to numeric
df[columns_to_clean] = df[columns_to_clean].apply(pd.to_numeric, errors='coerce')
keep = ['Size', 'Building SF', 'Number Of Units', 'Number Of Floors', 
         'Price Per AC Land', 'Price Per SF Land', 'Asking Price', 
         'Number Of 1 Bedrooms Units', 'Number Of 2 Bedrooms Units', 
         'Floor Area Ratio', 'Number Of Parking Spaces', 
         'Number Of Studios Units', 'Typical Floor (SF)', 
         'Number Of 3 Bedrooms Units', 'Land Area AC', 'Land Area SF', 
         'Star Rating', 'Net Income', 'Year Built', 'Age',
         'Sale Price']

df = df[keep]
df.dropna(axis = 0, inplace=True)


num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])



# Save the cleaned dataset
cleaned_file = '/data/cleaned_dataset.csv'
df.to_csv(cleaned_file, index=False)


df.head()

/var/folders/76/xf1kw1lj2915163138yqzp0h0000gn/T/ipykernel_22912/4031419765.py:24: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace('$', '')


,Size,Building SF,Number Of Units,Number Of Floors,Price Per AC Land,Price Per SF Land,Asking Price,Number Of 1 Bedrooms Units,Number Of 2 Bedrooms Units,Floor Area Ratio,...,Number Of Studios Units,Typical Floor (SF),Number Of 3 Bedrooms Units,Land Area AC,Land Area SF,Star Rating,Net Income,Year Built,Age,Sale Price
16,4872,4872.0,8.0,2.0,10044718.96,230.60,"1,075,000",3.0,0.0,1.00,...,5.0,2436.0,0.0,0.11,4857.0,3.0,47515.0,1929.0,94.0,1120000.0
26,8898,8898.0,7.0,2.0,27260869.57,625.82,"4,000,000",6.0,1.0,1.47,...,0.0,3131.0,0.0,0.14,6072.0,2.0,107160.0,1939.0,84.0,3800000.0
27,6474,6474.0,4.0,2.0,20078107.85,460.93,"2,915,000",0.0,0.0,1.05,...,4.0,2848.0,0.0,0.14,6194.0,2.0,108730.0,1931.0,92.0,2855000.0
33,3980,3980.0,5.0,2.0,14209048.36,326.20,"2,375,000",4.0,1.0,0.56,...,0.0,3153.0,0.0,0.16,7051.0,2.0,77633.0,1941.0,82.0,2300000.0
35,4798,4798.0,9.0,2.0,7695600.00,176.67,"1,435,000",1.0,0.0,0.64,...,8.0,2399.0,0.0,0.17,7500.0,2.0,66117.0,1924.0,99.0,1325000.0


In [20]:
selected_features = ['Size', 'Number Of Units', 'Number Of Floors', 
                     'Price Per AC Land',  
                     'Number Of 1 Bedrooms Units', 'Number Of 2 Bedrooms Units', 
                     'Floor Area Ratio', 'Number Of Parking Spaces', 
                     'Number Of Studios Units',
                     'Number Of 3 Bedrooms Units', 'Land Area AC', 
                     'Star Rating', 'Net Income', 'Year Built']
# removed: 'Building SF', 'Price Per SF Land', 'Typical Floor (SF)', 'Land Area SF', 'Age'


# setting train test data

df.dropna(axis = 0)

X = df[selected_features]
y = df['Sale Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)


# models

models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor()
}

num_cols = X_train.select_dtypes(include=['float64', 'int64']).columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols),
    ])


# run models

for name, model in models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)
                              ])
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mpe = np.mean(np.abs((y_test - y_pred)/y_test))*100

    print(f"Model: {name}")
    print(f"Mean Absolute Error: {mae}")
    print(f"Mean Squared Error: {mse}")
    print(f"R-squared: {r2}")
    print(f"Mean Percentage Error: {mpe}")
    print("="*40)





non_numeric_columns = X.select_dtypes(exclude=['number']).columns
X_train_numeric = X_train.drop(columns=non_numeric_columns)
X_test_numeric = X_test.drop(columns=non_numeric_columns)

poly = PolynomialFeatures(degree=2, interaction_only = 2, include_bias = False)

X_poly_train = poly.fit_transform(X_train_numeric)
X_poly_test = poly.transform(X_test_numeric)

rf = RandomForestRegressor(random_state = 42)
gb = GradientBoostingRegressor(random_state = 42)
high_performing_models = {
    'Random Forest': rf,
    'Gradient Boosting': gb
}

for name, model in high_performing_models.items():
    model.fit(X_poly_train, y_train)
    y_pred = model.predict(X_poly_test)
                           
    mpe = np.mean(np.abs((y_test - y_pred)/y_test))*100
    
    print(f"{name} - MAE: {mean_absolute_error(y_test, y_pred)}, MSE: {mean_squared_error(y_test, y_pred)}, R2: {r2_score(y_test, y_pred)}, MPE: {mpe}")

print("="*80)

Model: Linear Regression
Mean Absolute Error: 449169.802863303
Mean Squared Error: 479949166555.382
R-squared: 0.9589265816593304
Mean Percentage Error: 14.648396614723497
Model: Random Forest
Mean Absolute Error: 386078.7569736842
Mean Squared Error: 462080148250.3722
R-squared: 0.9604557887406684
Mean Percentage Error: 10.818906981950775
Model: Gradient Boosting
Mean Absolute Error: 396879.1308163576
Mean Squared Error: 500366334094.0816
R-squared: 0.9571793073184512
Mean Percentage Error: 9.82928676415957


/Users/lehergulati/anaconda3/lib/python3.11/site-packages/sklearn/utils/_param_validation.py:591: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(


Random Forest - MAE: 161528.13381578948, MSE: 165943554342.3835, R2: 0.9857987689042984, MPE: 2.5314391835461976
Gradient Boosting - MAE: 186386.53093156588, MSE: 157692916827.71655, R2: 0.9865048476097774, MPE: 3.7073513206821818


'\n    \n\nprint("Removing highly correlated values: \n\n")\n\nselected_features = [\'Size\', \'Number Of Units\', \'Number Of Floors\', \n                     \'Price Per AC Land\',  \n                     \'Number Of 1 Bedrooms Units\', \'Number Of 2 Bedrooms Units\', \n                     \'Floor Area Ratio\', \'Number Of Parking Spaces\', \n                     \'Number Of Studios Units\',\n                     \'Number Of 3 Bedrooms Units\', \'Land Area AC\', \n                     \'Star Rating\', \'Net Income\', \'Year Built\']\n\n\n\nthreshold_high_corr = 0.9\nhighly_corr_pairs = []\ncorr_matrix_features = df.corr()\n\nfor i in range(len(corr_matrix_features.columns)):\n    for j in range(i+1, len(corr_matrix_features.columns)):\n        if abs(corr_matrix_features.iloc[i, j]) > threshold_high_corr:\n            highly_corr_pairs.append((corr_matrix_features.columns[i], corr_matrix_features.columns[j], corr_matrix_features.iloc[i, j]))\n\nfeatures_to_remove = [pair[1] for pair